In [3]:
from IPython.display import Image, HTML

# Activation function

> **purpose of activation function?**

# ReLU and Softmax VS sigmoid, cross entropy vs SSE

- relu replace sigmoid on hidden layer for vanishing problem of sigmoid
- softmax replace sigmoid (binary classification) on multi-class classification on output layer
- cross entropy replace SSE (evaluate regression or binary classification) for multi-class classification evaluation

## ReLU activation function

----
sigmoid function can be used as activation function on our hidden units and on the output unit. 

----
> **Why sigmoids is not favored as activations on hidden units -- vanishing effect**
- the **derivative of the sigmoid** maxes out at 0.25 (see below). 
- error of hidden layers is calculated using output error or derivative of E
- meaning the errors going back into the network will be **shrunk by at least a quarter at every layer**
- For **layers close to the input layer**, the weight updates will be tiny if you have a lot of layers with sigmoid functions and those weights will **take a really long time to train**

In [2]:
Image(width=500, height=300, url='https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5893d15c_sigmoids/sigmoids.png')

# Rectified Linear Units
- a replacement to sigmoid function as activation function for hidden or output layers

> **rectified linear units (ReLUs)**
- Instead of sigmoids, **most recent deep learning networks use it for the hidden layers**. 
- A rectified linear unit has output 0 if the input is less than 0, and raw output otherwise. $$f(x)=\max(x,0)$$
- visually (below) is like combining $y=0$ for ($x \le 0$) and $y=x$ for ($x \gt 0$)

In [4]:
Image(width=300, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2017/February/58915ae8_relu/relu.png')

### **ReLU activations** 

- the simplest non-linear activation function
- When the **input is positive, the derivative is 1**, so **no vanishing effect** on backpropagated errors from sigmoids 
- ReLUs result in much **faster** training for large networks, simply there is **no shrinking of weight step size**, meaning it is not like sigmoid to slow down to 25% on each hidden layer. [based on research](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf). (see image below)
- No need **implementation** of relu ourselves in most frameworks like TensorFlow and TFLearn.

In [6]:
Image(width=500, height=300, url="https://cdn-enterprise.discourse.org/udacity/uploads/default/optimized/4X/f/2/3/f23ac5ba95ef03f8289f9ba558d5ede6c09b59ed_1_690x238.png")

## Drawbacks
- source drawbacks come from the image above, and can be triggered by large learning rate
- when $a \le 0$, $f(a) = \frac{\partial{f(a)}}{\partial{a}} = 0$, this cause no firing at this relu neuron in forward pass and zero weight step update through this relu neuron in backward pass

#### My summary and discussion of what ReLU drawbacks is on forum
- check [forum response on this](https://discussions.udacity.com/t/how-to-understand-relus-drawbacks-properly/217556?u=pystrategyexplorer)

**ReLU drawbacks is triggered by large learning rate and achieved in 3 step process**

----
> **step1: backward pass**
- If learning_rate was set very large, it leads to overshoot;
- overshoot result in negative gradients
- negative gradient with large learning rate, results in large negative weight update
- large negative weights update, may result many weights of the layer to be negative 

----
> **step2: Forward pass:**
- negative weights could make $a = \sum wx < 0$ (e.g., if we are talking about first hidden layer)
- when $a \le 0$, ReLU function $f(a) = max(0, a)$ will only return 0
- ReLU neural units return 0 meaning this neuron doesn't fire, doesn't activate, is dead

----
> **step3: Backward pass: **
- backprop won't stop at this ReLU neuron simply because this neuron was not firing
- when updating weights for this hidden ReLU neuron, we need hidden_delta;
- to calc hidden_delta, we need hidden_error * derivative of $f(a)$
- given $a \le 0$ (which caused ReLU dead in forward pass), derivative of $f(a)$ can only return zero
- therefore, weight step for updating is also zero. 
- So, there is no update of hidden layer weights in backward pass
- by now, this neuron doesn't fire and won't update its weights at all and forever, even though calculation process described above going through this neuron all the time. 
- this is neurons are (brain) dead but training (body) is still running

----
> Therefore, solution is to reduce learning_rate

### Descriptions about ReLU drawbacks from lesson and tutorials online

[course description on relu](https://classroom.udacity.com/nanodegrees/nd101/parts/2a9dba0b-28eb-4b0e-acfa-bdcf35680d90/modules/e0ad6969-6321-437a-8279-d9b20a7cd00a/lessons/cec79d13-b41a-4c61-9441-eada5b39c9e6/concepts/8ee4c905-fa9c-40a4-9fd7-427a155b81b4)

> drawbacks: 
- It's possible that a large gradient can set the weights such
- that a ReLU unit will always be 0 (meaning, relu unit is dead)
- a lot of computation will be wasted in training (meaning, compute a lot but return just 0s, so it is a waste)

----
> From Andrej Karpathy's CS231n course:
- Unfortunately, ReLU units can be fragile during training and can “die”. 
- a large gradient flowing through a ReLU neuron could cause the weights to update in such a way that the neuron will never activate on any datapoint again
- If this happens, then **the gradient flowing through the unit will forever be zero** from that point on. 
- the ReLU units can irreversibly die during training since they can get knocked off the data manifold. 
- you may find that as much as 40% of your network can be “dead” (i.e. neurons that never activate across the entire training dataset) if the learning rate is set too high. 

----
googled it and here is [something seemingly make some sense to me](https://www.reddit.com/r/MachineLearning/comments/3ij6nz/confused_about_why_relus_show_benefit_in_deep/)
> In back-propagation, you need to know the derivative of the non-linear function [my comment: which is relu, I think] to figure out how to change the weights before that function. If that derivative is zero [my comment: meaning gradient is zero], you've basically destroyed information.


----
> If you stare at this for a while you’ll see that if a neuron gets clamped to zero in the forward pass (i.e. z=0, it doesn’t “fire”), then its weights will get zero gradient. This can lead to what is called the “dead ReLU” problem, where if a ReLU neuron is unfortunately initialized such that it never fires, or if a neuron’s weights ever get knocked off with a large update during training into this regime, then this neuron will remain permanently dead. It’s like permanent, irrecoverable brain damage. Sometimes you can forward the entire training set through a trained network and find that a large fraction (e.g. 40%) of your neurons were zero the entire time.
[see detail in the article](https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b#.y4a08ce90)

![](https://cdn-images-1.medium.com/max/1600/1*g0yxlK8kEBw8uA1f82XQdA.png)



----
> solution: 
- With a proper setting of the learning rate this is less frequently an issue.

# Softmax: activation for multi-class problem

> **sigmoid on binary classification**
- ReLU can replace sigmoind as hidden layer activation function, but can't do classification on output layer
- sigmoid works fine for neural networks regression (bike riders?????) and binary classification
- however, sigmoind can't help **multi-class classification problem** 

----
> **softmax on multi-class classification**
- is **for multi-class classification problem** 
- The softmax function squashes the outputs of each unit to be between 0 and 1, just like a sigmoid. 
- It also divides each output such that the total sum of the outputs is equal to 1. 
- The output of the softmax function is equivalent to a categorical probability distribution, it tells you the probability that any of the classes are true.

----
> **How softmax work**
- if you have three inputs to a softmax function, say from a network with three output units, it'd look like:

In [7]:
Image(width=300, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2017/February/58950908_softmax-input-output/softmax-input-output.png')

----
> **Formula of Softmax**
- all I undertand is that it takes in a vector and return a vector
- the outputs are squashed and they sum to one
- for example, for network to classify 0-9 digits, for image of 4, it returns
- the output unit corresponding to 4 would be activated, and so on for the rest of the units
- seemingly no limit for number of classes

In [8]:
Image(width=300, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2017/February/58938e9e_softmax-math/softmax-math.png')

In [21]:
Image(width=300, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2017/February/58925c7f_softmax/softmax.png')

# Categorical Cross-Entropy
- cost function for multi-class classification

----
> **SSE as cost function: **
- using the sum of squared errors as the cost function in our networks,
- when expecting only singular (scalar) output values.

----
> **multi-class classification problems: **
- using softmax
- output is a vector of probability values from the output units
- **data labels** can be expressed as a vector using **one-hot encoding**

----
> **How to express predictions and labels**
- you have a vector the length of the number of classes
- the label element is marked with a 1 while the other labels are set to 0. 
- our label vector for the image of the number 4 would be: $$ y=[0,0,0,0,1,0,0,0,0,0]$$
- our output prediction vector could be something like $$ \hat{y}=[0.047,0.048,0.061,0.07,0.330,0.062,0.001,0.213,0.013,0.150]$$

----
> **We want our error to be proportional to how far apart these vectors are**
- To calculate this distance, use the cross entropy
- our goal: to make our prediction vectors as close as possible to the label vectors by minimizing the cross entropy. 

----
> **The cross entropy calculation:**

In [36]:
Image(width=300, height=100, url='https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5893b106_cross-entropy-diagram/cross-entropy-diagram.png')

----
> **Calculate cross entropy**: 
- the sum of the label elements times the (natural) log of the prediction probabilities
- this formula is not symmetric
- Flipping the vectors is a bad idea because the label vector has a lot of zeros and taking the log of zero will cause an error

----
> **What's cool about using one-hot encoding for the label vector**
- $y_j$ is 0 except for the one true class 
- all terms in that sum except for where $y_j=1$ are zero 
- the cross entropy is simply $D=−\log{\hat{y}}$ for the true label
- if your input image is of the digit 4 and it's labeled 4, then only the output of the unit corresponding to 4 matters in the cross entropy cost.

----
> **summary:**
- cross entropy is multi-class problem cost function
- calc cross entropy using $D(\hat{y}, y) = - \sum_j y_j\log{\hat{y_j}}$
- this forumla can be simplified: $D(\hat{y}, y) = E = -\log{\hat{y_j}}$
- whereas $ SSE = E = \frac{1}{2}\sum(y-\hat{y})^2 $


QUIZ QUESTION

If your label vector is [0, 0, 0, 1, 0] and the predicted probabilities are [0.27, 0.11, 0.33, 0.10, 0.19], what is the cross entropy?

In [37]:
import numpy as np
E = - np.log(0.1)
E

2.3025850929940455